## Setup

In [1]:
import torch
import numpy as np
from torch import einsum
from tqdm.auto import tqdm
import seaborn as sns
from transformer_lens import HookedTransformer, ActivationCache, utils, patching
from datasets import load_dataset
from einops import einsum
import pandas as pd
from transformer_lens import utils
from rich.table import Table, Column
from rich import print as rprint
from jaxtyping import Float, Int, Bool
from typing import List, Tuple
from torch import Tensor
import einops
import functools
from transformer_lens.hook_points import HookPoint
# import circuitsvis
from IPython.display import HTML
from plotly.express import line
import plotly.express as px
from tqdm.auto import tqdm
import json
import gc
import plotly.graph_objects as go

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from plotly.subplots import make_subplots
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

from haystack_utils import load_txt_data, get_mlp_activations, line
import haystack_utils

%reload_ext autoreload
%autoreload 2

In [2]:
english_neurons = [(5, 395), (5, 166), (5, 908), (5, 285), (3, 862), (5, 73), (4, 896), (5, 348), (5, 297), (3, 1204)]
german_neurons = [(4, 482), (5, 1039), (5, 407), (5, 1516), (5, 1336), (4, 326), (5, 250), (3, 669)]
french_neurons = [(5, 112), (4, 1080), (5, 1293), (5, 455), (5, 5), (5, 1901), (5, 486), (4, 975)]

model = HookedTransformer.from_pretrained("EleutherAI/pythia-70m",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device)

english_data = haystack_utils.load_txt_data("kde4_english.txt")
german_data = haystack_utils.load_txt_data("wmt_german_large.txt")

english_activations = {}
german_activations = {}
for layer in range(3, 6):
    english_activations[layer] = get_mlp_activations(english_data[:200], layer, model, mean=False)
    german_activations[layer] = get_mlp_activations(german_data[:200], layer, model, mean=False)

LOG_PROB_THRESHOLD = -7
LAYER_TO_ABLATE = 3
NEURONS_TO_ABLATE = [669]
MEAN_ACTIVATION_ACTIVE = german_activations[LAYER_TO_ABLATE][:, NEURONS_TO_ABLATE].mean()
MEAN_ACTIVATION_INACTIVE = english_activations[LAYER_TO_ABLATE][:, NEURONS_TO_ABLATE].mean()

def deactivate_neurons_hook(value, hook):
    value[:, :, NEURONS_TO_ABLATE] = MEAN_ACTIVATION_INACTIVE
    return value
deactivate_neurons_fwd_hooks=[(f'blocks.{LAYER_TO_ABLATE}.mlp.hook_post', deactivate_neurons_hook)]

    
def activate_neurons_hook(value, hook):
    value[:, :, NEURONS_TO_ABLATE] = MEAN_ACTIVATION_ACTIVE
    return value
activate_neurons_fwd_hooks=[(f'blocks.{LAYER_TO_ABLATE}.mlp.hook_post', activate_neurons_hook)]

all_ignore, not_ignore = haystack_utils.get_weird_tokens(model, plot_norms=False)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
kde4_english.txt: Loaded 1007 examples with 501 to 5295 characters each.
wmt_german_large.txt: Loaded 2459 examples with 800 to 2000 characters each.


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [3]:
def get_pos_loss_diff(prompt: str, model: HookedTransformer, activate_neurons_hook: List[Tuple[str, HookPoint]], deactivate_neurons_hook: List[Tuple[str, HookPoint]], plot_hist=False, use_activate_hook=False, debug_log=True):
    tokens = model.to_tokens(prompt)
    if use_activate_hook:
        original_loss = model.run_with_hooks(tokens, return_type="loss", fwd_hooks=activate_neurons_hook, loss_per_token=True)
    else:
        original_loss = model(tokens, return_type="loss", loss_per_token=True)
    ablated_loss = model.run_with_hooks(tokens, return_type="loss", fwd_hooks=deactivate_neurons_hook, loss_per_token=True)
    
    # Positive difference = loss increase due to ablation
    loss_difference = (ablated_loss - original_loss).flatten()

    if debug_log:
        print(f"Unablated loss: {original_loss.flatten()}")
        print(f"Ablated loss: {ablated_loss.flatten()}")
        print(f"Loss difference: {loss_difference}")

    if plot_hist:
        fig = px.histogram(loss_difference.flatten().cpu().numpy(), title="Loss difference due to ablation per position")
        fig.show()
    return loss_difference

def get_high_loss_prompts(prompts: list[str], model: HookedTransformer, activate_neurons_hook: List[Tuple[str, HookPoint]], deactivate_neurons_hook: List[Tuple[str, HookPoint]]):
    max_diffs = []
    average_diffs = []
    for prompt in tqdm(prompts):
        loss_difference = get_pos_loss_diff(prompt, model, activate_neurons_hook, deactivate_neurons_hook)
        max_diffs.append(loss_difference.max().item())
        average_diffs.append(loss_difference.mean().item())
    return max_diffs, average_diffs


In [4]:
def get_top_differences_at_position(prompt: str, model: HookedTransformer, position: int, top_k=20, mode="full"):
    tokens = model.to_tokens(prompt)
    str_tokens = model.to_str_tokens(tokens)
    # Logprobs instead of logits
    original_logits = model(tokens, return_type="logits")
    if mode=="direct":
        to_freeze = ["blocks.4.hook_attn_out", "blocks.5.hook_attn_out", "blocks.4.hook_mlp_out", "blocks.5.hook_mlp_out"]
        ablated_logits = haystack_utils.get_frozen_logits(prompt, model, ablation_hooks=deactivate_neurons_fwd_hooks, freeze_act_names=to_freeze)
    elif mode=="indirect":
        to_freeze = ["blocks.4.hook_attn_out", "blocks.5.hook_attn_out", "blocks.4.hook_mlp_out", "blocks.5.hook_mlp_out"]
        ablated_logits = haystack_utils.get_ablated_logits(prompt, model, ablation_hooks=deactivate_neurons_fwd_hooks, freeze_act_names=to_freeze)
    else:
        assert mode=="full"
        ablated_logits = model.run_with_hooks(tokens, return_type="logits", fwd_hooks=deactivate_neurons_fwd_hooks)
    original_logprob = original_logits.log_softmax(dim=-1)
    ablated_logprob = ablated_logits.log_softmax(dim=-1)

    # Positive difference = the German neuron makes the token more likely
    # Negative difference = the German neuron makes the token less likely
    logprob_differences = original_logprob - ablated_logprob
    logit_differences = original_logits - ablated_logits

    print("Prompt:", prompt)
    print(f"Differences for predicting: {str_tokens[position]} -> {str_tokens[position+1]}")

    low_log_prob = torch.argwhere(((original_logprob[0, position, :] <= LOG_PROB_THRESHOLD) & (ablated_logprob[0, position, :] <= LOG_PROB_THRESHOLD))).flatten()
    ignore_tokens = torch.cat([low_log_prob, all_ignore]).unique()
    
    top_original_logprobs, top_original_idx = haystack_utils.top_k_with_exclude(original_logprob[0, position, :].flatten(), top_k, exclude=ignore_tokens)
    top_original_ablated_logprobs = ablated_logprob[0, position, top_original_idx]
    top_ablated_logprobs, top_ablated_idx = haystack_utils.top_k_with_exclude(ablated_logprob[0, position, :].flatten(), top_k, exclude=ignore_tokens)
    top_ablated_original_logprobs = original_logprob[0, position, top_ablated_idx]

    top_original_values = logprob_differences[0, position, top_original_idx]
    top_ablated_values = logprob_differences[0, position, top_ablated_idx]
    top_original_logit_diff = logit_differences[0, position, top_original_idx]
    top_ablated_logit_diff = logit_differences[0, position, top_ablated_idx]
    print("Top predictions with German neuron active (unablated)")
    haystack_utils.print_strings_as_html(model.to_str_tokens(top_original_idx), top_original_values.cpu().tolist(), max_value=5, original_log_probs=top_original_logprobs.cpu().tolist(), ablated_log_probs=top_original_ablated_logprobs.cpu().tolist(), logit_difference=top_original_logit_diff.cpu().tolist())
    print("Top predictions with German neuron disabled (ablated)")
    haystack_utils.print_strings_as_html(model.to_str_tokens(top_ablated_idx), top_ablated_values.cpu().tolist(), max_value=5, original_log_probs=top_ablated_original_logprobs.cpu().tolist(), ablated_log_probs=top_ablated_logprobs.cpu().tolist(), logit_difference=top_ablated_logit_diff.cpu().tolist())

    top_boosts, top_boosted_idx = haystack_utils.top_k_with_exclude(logprob_differences[:, position, :].flatten(), top_k, exclude=ignore_tokens)
    top_boost_original_logprob = original_logprob[0, position, top_boosted_idx]
    top_boost_ablated_logprob = ablated_logprob[0, position, top_boosted_idx]
    top_reduced, top_reduced_idx = haystack_utils.top_k_with_exclude(logprob_differences[:, position, :].flatten(), top_k, largest=False, exclude=ignore_tokens)
    top_reduced_original_logprob = original_logprob[0, position, top_reduced_idx]
    top_reduced_ablated_logprob = ablated_logprob[0, position, top_reduced_idx]
    print("Top boosted tokens by German neuron")
    haystack_utils.print_strings_as_html(model.to_str_tokens(top_boosted_idx), top_boosts.cpu().tolist(), max_value=5, original_log_probs=top_boost_original_logprob.cpu().tolist(), ablated_log_probs=top_boost_ablated_logprob.cpu().tolist())
    print("Top reduced tokens by German neuron")
    haystack_utils.print_strings_as_html(model.to_str_tokens(top_reduced_idx), top_reduced.cpu().tolist(), max_value=5, original_log_probs=top_reduced_original_logprob.cpu().tolist(), ablated_log_probs=top_reduced_ablated_logprob.cpu().tolist())

In [15]:
def show_token_loss(prompt: str, model: HookedTransformer, max_value=None, mode="full", freeze_act_names=("blocks.4.hook_attn_out", "blocks.5.hook_attn_out", "blocks.4.hook_mlp_out", "blocks.5.hook_mlp_out"), custom_loss_change=None):
    
    original_loss, total_effect_loss_change, direct_effect_loss_change, indirect_effect_loss_change = haystack_utils.split_effects(prompt, model, ablation_hooks=deactivate_neurons_fwd_hooks, freeze_act_names=freeze_act_names, debug_log=False)
    if mode== "indirect":
        pos_wise_loss = indirect_effect_loss_change
        #pos_wise_loss = haystack_utils.get_frozen_loss_difference_measure(prompt, model, ablation_hooks=deactivate_neurons_fwd_hooks, freeze_act_names=freeze_act_names)
    elif mode == "direct":
        pos_wise_loss = direct_effect_loss_change
        #pos_wise_loss = haystack_utils.get_ablated_loss_difference_measure(prompt, model, ablation_hooks=deactivate_neurons_fwd_hooks, freeze_act_names=freeze_act_names)
    elif mode == "custom":
        pos_wise_loss = custom_loss_change
    else:
        assert mode =="full"
        pos_wise_loss = total_effect_loss_change
        #pos_wise_loss = get_pos_loss_diff(prompt, model, activate_neurons_fwd_hooks, deactivate_neurons_fwd_hooks, plot_hist=False)
    str_token_prompt = model.to_str_tokens(model.to_tokens(prompt))
    haystack_utils.print_strings_as_html(str_token_prompt[1:], pos_wise_loss.flatten().cpu().tolist(), max_value=max_value)

def print_predictions(prompt, pos, k=20):
    print("\nFull model predictions")
    get_top_differences_at_position(prompt, model, pos, k, mode="full")
    print("\nIndirect predictions (leave German neuron active, patch corrupted activations to later components)")
    get_top_differences_at_position(prompt, model, pos, k, mode="indirect")
    print("\nDirect predictions (ablate German neuron, patch clean activations to later components)")
    get_top_differences_at_position(prompt, model, pos, k, mode="direct")

In [8]:
haystack_utils.pos_wise_mlp_effect_on_single_prompt("Ich möchte nochmals meine Ansicht", model, deactivate_neurons_fwd_hooks, k=20, log=True, answer_pos=None)


Ich möchte nochmals meine Ansicht
Original loss: tensor([10.5044,  5.3246,  0.0203,  4.4463,  3.9146,  0.4182,  6.2384,  2.9002,
         9.2651,  4.7154], device='cuda:0')
Total effect loss: tensor([10.4859,  6.1975,  0.0941,  5.4040,  4.3693,  1.0097,  6.9663,  4.0273,
        10.5419, 12.9027], device='cuda:0')
Direct effect loss of MLP3 and MLP5 (restoring MLP4 and attention): tensor([10.4888,  5.9200,  0.0217,  4.7280,  3.5013,  0.6890,  6.1393,  2.8607,
         9.1133,  7.9633], device='cuda:0')
Direct effect loss of MLP3 (restoring MLP4 and MLP5 and attention): tensor([10.4970,  5.8691,  0.0374,  5.2219,  2.9650,  0.2496,  6.1509,  2.6764,
         9.3340,  3.3191], device='cuda:0')
Total effect loss when freezing top MLP5 neurons: tensor([10.4843,  5.7237,  0.0467,  4.6787,  2.7740,  0.0540,  5.3758,  1.9933,
         9.5463,  7.1096], device='cuda:0')


(tensor([10.5044,  5.3246,  0.0203,  4.4463,  3.9146,  0.4182,  6.2384,  2.9002,
          9.2651,  4.7154], device='cuda:0'),
 tensor([10.4859,  6.1975,  0.0941,  5.4040,  4.3693,  1.0097,  6.9663,  4.0273,
         10.5419, 12.9027], device='cuda:0'),
 tensor([10.4888,  5.9200,  0.0217,  4.7280,  3.5013,  0.6890,  6.1393,  2.8607,
          9.1133,  7.9633], device='cuda:0'),
 tensor([10.4970,  5.8691,  0.0374,  5.2219,  2.9650,  0.2496,  6.1509,  2.6764,
          9.3340,  3.3191], device='cuda:0'),
 tensor([10.4843,  5.7237,  0.0467,  4.6787,  2.7740,  0.0540,  5.3758,  1.9933,
          9.5463,  7.1096], device='cuda:0'))

In [11]:
original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt("Ich möchte nochmals meine Ansicht", model, deactivate_neurons_fwd_hooks, k=20, log=False, answer_pos=-1)

list_answers = [[x] for x in [original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss]]
names = ["Original loss", "Ablated Loss", "Ablated loss (restoring MLP4)", "Ablated loss (restoring MLP4 + MLP5)", "Ablated loss (restoring top MLP5 neurons)"]
haystack_utils.plot_barplot(list_answers, names)

## Look for interesting examples

In [33]:
def custom_diff_weight(total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss):
    # We want high loss difference between total effect and direct effect of the context neuron (so restoring MLP4 and MLP5 fixes the problem)
    indirect_diff = total_effect_loss - direct_mlp3_loss
    # We want high loss difference between frozen MLP4 and the direct effect of the context neuron (so only restoring MLP4 is not enough)
    mlp_4_diff = direct_mlp3_mlp5_loss - direct_mlp3_loss
    # We want high loss difference between frozen MLP5 and the direct effect of the context neuron (so only restoring MLP5 is not enough)
    mlp_5_diff = frozen_loss - direct_mlp3_loss
    return indirect_diff, mlp_4_diff, mlp_5_diff

In [46]:
def get_interesting_loss_prompts(prompts: list[str], model: HookedTransformer):
    max_indirect_diff, max_mlp_4_diff, max_mlp_5_diff = [], [], []
    for prompt in tqdm(prompts):
        original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=50, log=False, answer_pos=None)
        indirect_diff, mlp_4_diff, mlp_5_diff = custom_diff_weight(total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss)
        max_indirect_diff.append(indirect_diff.max().item())
        max_mlp_4_diff.append(mlp_4_diff.max().item())
        max_mlp_5_diff.append(mlp_5_diff.max().item())
    return max_indirect_diff, max_mlp_4_diff, max_mlp_5_diff

n_examples = 1000
max_indirect_diff, max_mlp_4_diff, max_mlp_5_diff = get_interesting_loss_prompts(german_data, model)

  0%|          | 0/2459 [00:00<?, ?it/s]

In [45]:
# Filter for examples with high difference in top MLP5 neurons and MLP4+MLP5 - these are the examples where patching some MLP5 is not enough (so maybe they rely on MLP4)
loss_data_tuple = [(diff, example) for diff, example in zip(max_mlp_5_diff, german_data[:n_examples])]
loss_data_tuple.sort(key=lambda x: x[0], reverse=True)
loss_data_tuple[:2]

[(3.359272003173828,
  'Der Vorschlag zur Änderung dieser Verordnung über die Anwendung des dem Vertrag zur Gründung der Europäischen Gemeinschaft beigefügten Protokolls über das Verfahren bei einem übermäßigen Defizit wirft eine begründete Frage hinsichtlich der Anpassung des Systems Volkswirtschaftlicher Gesamtrechnungen ESVG 95 auf, das, wie Sie wissen, das vorherige, 1979 eingeführte Rechnungssystem ersetzt. Vor allem wirft er eine Frage auf zum Auftreten neuer Kategorien von Finanzprodukten, zur Kohärenz bei der Berechnung von Proportionen zwischen dem öffentlichen Defizit und dem Bruttoinlandsprodukt im Rahmen dieses Systems Volkswirtschaftlicher Gesamtrechnungen von 1995 und schließlich zur Berücksichtigung der Bedeutung der Berechnung der "Zinszahlungen " und ihrer Kohärenz entsprechend dieser neuen Methodik.'),
 (3.2902698516845703,
  'Ziel ist die Einrichtung eines Informationssystems, das ein Echtzeit-Verkehrsmanagement und den Einsatz elektronischer Transaktionen im Güterve

In [39]:
for _ , prompt in loss_data_tuple[0:2]:
    print(f"\n{prompt}")
    original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=20, log=False, answer_pos=None)    
    mlp_5_diff = frozen_loss - direct_mlp3_loss
    mlp_4_diff = direct_mlp3_mlp5_loss - direct_mlp3_loss
    show_token_loss(prompt, model, max_value=5, mode="custom", custom_loss_change=mlp_4_diff)
    show_token_loss(prompt, model, max_value=5, mode="custom", custom_loss_change=mlp_5_diff)
    show_token_loss(prompt, model, max_value=5, mode="full")
    show_token_loss(prompt, model, max_value=5, mode="indirect")
    show_token_loss(prompt, model, max_value=5, mode="direct")


Der Vorschlag zur Änderung dieser Verordnung über die Anwendung des dem Vertrag zur Gründung der Europäischen Gemeinschaft beigefügten Protokolls über das Verfahren bei einem übermäßigen Defizit wirft eine begründete Frage hinsichtlich der Anpassung des Systems Volkswirtschaftlicher Gesamtrechnungen ESVG 95 auf, das, wie Sie wissen, das vorherige, 1979 eingeführte Rechnungssystem ersetzt. Vor allem wirft er eine Frage auf zum Auftreten neuer Kategorien von Finanzprodukten, zur Kohärenz bei der Berechnung von Proportionen zwischen dem öffentlichen Defizit und dem Bruttoinlandsprodukt im Rahmen dieses Systems Volkswirtschaftlicher Gesamtrechnungen von 1995 und schließlich zur Berücksichtigung der Bedeutung der Berechnung der "Zinszahlungen " und ihrer Kohärenz entsprechend dieser neuen Methodik.



Ziel ist die Einrichtung eines Informationssystems, das ein Echtzeit-Verkehrsmanagement und den Einsatz elektronischer Transaktionen im Güterverkehr ermöglichen soll, und zwar durch folgende Maßnahmen: Entwicklung intermodaler elektronischer Echtzeit-Transaktions- und Informationssysteme; Harmonisierung der Kommunikationsstandards, der Verfahren und der Transportdokumente auf EDI-Grundlage; Vereinheitlichung der Haftungsregelung mit dem Ziel, dem Kunden ein klares Regelwerk transparenter Haftungsbedingungen und -verfahren für beschädigte oder unterwegs abhanden gekommene Frachten bieten zu können; Entwicklung der Technologien der Informationsgesellschaft zum Nutzen des intermodalen Verkehrs durch die Schaffung einer offenen Systemarchitektur, die eine intermodale Ortung und Laufverfolgung (tracking and tracing) ermöglicht; Ermöglichung des papierlosen Verkehrs durch Harmonisierung der vorgeschriebenen Dokumente und Entwicklung einer einzigen elektronischen Anlaufstelle.


In [44]:
prompt = "Ich möchte nochmals meine Ansicht"
prompt = "Der Vorschlag zur Änderung dieser Verordnung über die Anwendung des dem Vertrag"
k = 20
# Check loss MLP5 loss increase when patching clean activations to MLP4
original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=k, log=False, answer_pos=-1)

list_answers = [[x] for x in [original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss]]
names = ["Original loss", "Ablated Loss", "Ablated loss (restoring MLP4)", "Ablated loss (restoring MLP4 + MLP5)", "Ablated loss (restoring top MLP5 neurons)"]
haystack_utils.plot_barplot(list_answers, names, title=f"Losses on pos={-1} for different frozen components (MLP5: top {k} neurons)")

## Superposition on "Vertrag"
- The model needs ~100 neurons to restore the loss

In [43]:
prompt = "Der Vorschlag zur Änderung dieser Verordnung über die Anwendung des dem Vertrag"
k = 100
# Check loss MLP5 loss increase when patching clean activations to MLP4
original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss = haystack_utils.pos_wise_mlp_effect_on_single_prompt(prompt, model, deactivate_neurons_fwd_hooks, k=k, log=False, answer_pos=-1)

list_answers = [[x] for x in [original_loss, total_effect_loss, direct_mlp3_mlp5_loss, direct_mlp3_loss, frozen_loss]]
names = ["Original loss", "Ablated Loss", "Ablated loss (restoring MLP4)", "Ablated loss (restoring MLP4 + MLP5)", "Ablated loss (restoring top MLP5 neurons)"]
haystack_utils.plot_barplot(list_answers, names, title=f"Losses on pos={-1} for different frozen components (MLP5: top {k} neurons)")